In [1]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
username = "neo4j"
password = "password"
driver = GraphDatabase.driver(uri, auth=(username, password))

In [2]:
try:
    with driver.session() as session:
        result = session.run("RETURN 1 AS test")
    print("Connection successful")
except Exception as e:
    print(f"Connection failed: {e}")

Connection successful


In [3]:
import os
with open('./pathCSV.txt','r') as f:
    path = f.read()
os.path.exists(path)

True

In [4]:
business_path = "business.csv"
review_path = "review.csv"
# tip_path = "yelp_academic_dataset_tip.json"
user_path = "users.csv"

business_path, review_path, user_path = list(map(lambda y: path + '/' + y,[business_path, review_path, user_path]))
# business_path, review_path, user_path = list(map(lambda y:'"'+ path + '/' + y+'"',[business_path, review_path, user_path]))
print(business_path)

../filteredCSV/business.csv


In [ ]:
def import_csv(tx, csv_file_path, query):
    with open(csv_file_path, 'r') as file:
        tx.run(query, file=file)
    # with open(csv_file_path, 'r') as file:
    #     csv_data = file.read()
    # tx.run(query,file=file)
# csv_file_path = "path/to/your/data.csv"

import pandas as pd


# Function to create nodes and relationships
def create_graph(tx, data):
    # for _, row in df.iterrows():
        # query = """CREATE (n:Businses  {name:$name,
        #change this to use SET so that ubusiness ID is only thing that is unique that is tehre
    query = """ UNWIND $business AS row
            Create (n:Businsess  {business_id:row.business_id})
            SET n.name=row.name, 
                n.address=row.address,
                n.city=row.city,
                n.state=row.state,
                n.latitude=row.latitude,
                n.longitude=row.longitude,
                n.stars=row.stars,
                n.review_count=row.review_count,
                n.hours=row.hours,
                n.attributes=row.attributes
            With n, row
            Merge (k:City {name: row.city, state: row.state})
            Merge (s:State {name:row.state})
            Merge (n)-[:IN_CITY]->(k)
            Merge (k)-[:IN_STATE]->(s)
            With n, split(row.categories,', ') as cats
            unwind cats AS cat
            merge (c:Category {name: cat})
            Merge (n)-[:IN_CATEGORY]->(c)
                """

    tx.run(query, business=data)

chunk_size = 10000
i=1
for chunk in pd.read_csv(business_path, chunksize=chunk_size):
    print(f"chunk {i} being processed")
    users= chunk.to_dict('records')
    print("chunk converted to dicts")
    with driver.session() as session:
        session.execute_write(create_graph, users)
    print("chunk processed")
    i+=1

# Close the driver
# driver.close()

chunk 1 being processed
chunk converted to dicts
chunk processed
chunk 2 being processed
chunk converted to dicts
chunk processed
chunk 3 being processed
chunk converted to dicts
chunk processed
chunk 4 being processed
chunk converted to dicts
chunk processed


In [ ]:
import pandas as pd

# Function to create nodes and relationships
def create_graph(tx, users_data):
    # for _, row in users.iterrows():
    # use create instead of merge for faster 
    query = """UNWIND $users AS row
            Merge (n:User {user_id: row.user_id})
                SET n.name = row.name,
                    n.review_count = row.review_count,
                    n.yelping_since = row.yelping_since,
                    n.useful = row.useful,
                    n.funny = row.funny,
                    n.cool = row.cool,
                    n.fans = row.fans,
                    n.elite= row.elite,
                    n.average_stars = row.average_stars,
                    n.compliment_hot = row.compliment_hot,
                    n.compliment_more = row.compliment_more,
                    n.compliment_profile = row.compliment_profile,
                    n.compliment_cute = row.compliment_cute,
                    n.compliment_list = row.compliment_list,
                    n.compliment_note = row.compliment_note,
                    n.compliment_plain = row.compliment_plain,
                    n.compliment_cool = row.compliment_cool,
                    n.compliment_funny = row.compliment_funny,
                    n.compliment_writer = row.compliment_writer
            WITH n, row.friends AS frens
            where frens = toString(frens)
                UNWIND split(frens,",") AS friend_id
                Merge (f:User {user_id: friend_id})
                MERGE (n)-[:HAS_FRIEND]->(f)
                """
    tx.run(query,users=users_data)

# Execute the function within a transaction
chunk_size = 5000
i=1
for chunk in pd.read_csv(user_path, chunksize=chunk_size):
    if i>4: #already loaded 4 chunks
        print(f"chunk {i} being processed")
        users= chunk.to_dict('records')
        print("chunk converted to dicts")
        with driver.session() as session:
            session.execute_write(create_graph, users)
        print("chunk processed")
    i+=1
# Close the driver
# driver.close()

chunk 5 being processed
chunk converted to dicts
chunk processed
chunk 6 being processed
chunk converted to dicts
chunk processed
chunk 7 being processed
chunk converted to dicts
chunk processed
chunk 8 being processed
chunk converted to dicts
chunk processed
chunk 9 being processed
chunk converted to dicts
chunk processed
chunk 10 being processed
chunk converted to dicts


In [ ]:
# Function to create nodes and relationships
def create_graph(tx, users_data):
    # for _, row in users.iterrows():
    # use create instead of merge for faster 
    query = """UNWIND $reviews AS row
            CREATE (n:Review {review_id: row.review_id})
                SET n.user_id = row.user_id,
                    n.business_id = row.business_id,
                    n.stars = row.stars,
                    n.useful = row.useful,
                    n.funny = row.funny,
                    n.cool = row.cool,
                    n.date = row.date,
                    n.text = row.text
            With n, n.user_id as user, n.business_id as business
                Match (u:User {user_id:user})
                Match (b:Business {business_id:business})
                MERGE (u)-[rel:WRITES_REVIEW]->(n)
                MERGE (n)-[rel:REVIEW_FOR]->(b)
                
                """
    tx.run(query,reviews=users_data)

# Execute the function within a transaction
chunk_size = 10000
i=1
for chunk in pd.read_csv(review_path, chunksize=chunk_size):
    print(f"chunk {i} being processed")
    reviews= chunk.to_dict('records')
    print("chunk converted to dicts")
    with driver.session() as session:
        session.execute_write(create_graph, reviews)
    print("chunk processed")
    i+=1